In [13]:
import os
import shutil
import glob
import json
import pickle
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import cv2
import skimage.io as io
from skimage.transform import resize
import scipy
from skimage.filters import sobel
from skimage.feature import graycomatrix , graycoprops
from skimage.measure import shannon_entropy
import lightgbm as lgbm
from imblearn.over_sampling import SMOTE
import tensorflow
from PIL import Image
!pip install --upgrade scikit-learn

from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.utils import shuffle

from sklearn.utils import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator, save_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential, Model, load_model, model_from_json
from tensorflow.keras.layers import (Input, InputLayer, Conv2D, MaxPooling2D, Dense, Concatenate, BatchNormalization, Dropout, Activation, GlobalAveragePooling2D, InputSpec, Flatten, Concatenate)
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.resnet_v2 import ResNet101V2, preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.metrics import Accuracy, AUC, Precision, Recall
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import sklearn
print(sklearn.__version__)

1.2.2


In [15]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [16]:
!pip install natsort
from natsort import natsorted

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from google.colab import drive
drive.mount('/content/drive')


ROOT_DIR = "/content/drive/MyDrive/brain_tumor_dataset/Br35H-Mask-RCNN"
number_of_images = {}

#This code lists the Amounts of Healthy and Non-healthy Brain Tumor Datasets
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR,dir)))

number_of_images.items()

Mounted at /content/drive


dict_items([('TEST', 101), ('TRAIN', 501), ('VAL', 202)])

In [8]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) 
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

'''
Initial and constant parameters for the model
initial parameters
       batch_size    : the initial batch size we will create our base model with
Constant parameters
       target_size   : Constant image target size for the whole notebook which is (224,224)
       class_mode    : class mode we will use for the whole notebook, which is 'categorical'
'''
# constant parameters
targetsize_constant = (224,224)
classmode_constant = 'categorical'

# initial parameters
batchsize_initial = 8
epochs_initial = 4
dropout_value_initial = 0.5
optimizer_initial = 'Adam'

# Data paths
train_dataset_path = '/content/drive/MyDrive/brain_tumor_dataset/Br35H-Mask-RCNN/TRAIN/'
validation_dataset_path = '/content/drive/MyDrive/brain_tumor_dataset/Br35H-Mask-RCNN/VAL/'
test_dataset_path = '/content/drive/MyDrive/brain_tumor_dataset/Br35H-Mask-RCNN/TEST/'


train_generator_initial = train_datagen.flow_from_directory(train_dataset_path, # this is where you specify the path to the main data folder
                                                 target_size=targetsize_constant,
                                                 color_mode='rgb',
                                                 batch_size=batchsize_initial,
                                                 class_mode=classmode_constant,
                                                 shuffle=True)
val_generator_initial = val_datagen.flow_from_directory(validation_dataset_path, # this is where you specify the path to the main data folder
                                                 target_size=targetsize_constant,
                                                 color_mode='rgb',
                                                 batch_size=batchsize_initial,
                                                 class_mode=classmode_constant,
                                                 shuffle=True)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil

IMG_PATH = '/content/drive/MyDrive/brain_tumor_dataset/'
TRAIN_DIR = 'TRAIN/'
VAL_DIR = 'VAL/'
TEST_DIR = 'TEST/'

# create directories if they don't exist
for dir in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    if not os.path.exists(dir):
        os.makedirs(dir)
    for CLASS in os.listdir(IMG_PATH):
        if not CLASS.startswith('.'):
            if not os.path.exists(dir + CLASS.upper()):
                os.makedirs(dir + CLASS.upper())

# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < 5:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
            elif n < 0.8*IMG_NUM:
                shutil.copy(img, 'TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'VAL/'+ CLASS.upper() + '/' + FILE_NAME)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IsADirectoryError: ignored

In [ ]:
IMG_PATH = '/content/drive/MyDrive/brain_tumor_dataset'
# split the data by train/val/test
for CLASS in CLASS_list:
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + '/' + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + '/' + CLASS)):
            img = IMG_PATH + '/' + CLASS + '/' + FILE_NAME
            if n < 50:
                shutil.copy(img, 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
            elif n < 0.8*IMG_NUM:
                shutil.copy(img, 'TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'VAL/'+ CLASS.upper() + '/' + FILE_NAME)

In [ ]:
IMG_PATH = '/content/drive/MyDrive/brain_tumor_dataset'
for CLASS in CLASS_list:
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + '/' + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + '/' + CLASS)):
            img = IMG_PATH + '/' + CLASS + '/' + FILE_NAME
            if n < 500:
                shutil.copy(img, 'TEST_BMI/' + CLASS.upper() + '/' + FILE_NAME)
            

In [ ]:
IMG_PATH = '/content/drive/MyDrive/brain_tumor_dataset'
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + '/' + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + '/' + CLASS)):
            img = IMG_PATH + '/' + CLASS + '/' + FILE_NAME
            if n < 500 and CLASS == 'Brain Tumor' :
                shutil.copy(img, 'TEST_BTD/' + 'YES' + '/' + FILE_NAME)
            elif n < 500 and CLASS == 'Healthy' :
                shutil.copy(img, 'TEST_BTD/' + 'NO' + '/' + FILE_NAME)  

In [ ]:
IMG_PATH = '/content/drive/MyDrive/brain_tumor_dataset'
for CLASS in CLASS_list:
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + '/' + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + '/' + CLASS)):
            img = IMG_PATH + '/' + CLASS + '/' + FILE_NAME
            if n < 500:
                shutil.copy(img, 'TEST_dat/' + CLASS.upper() + '/' + FILE_NAME)

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/brain_tumor_dataset/no')))
print(len(os.listdir('/content/drive/MyDrive/brain_tumor_dataset/yes')))
print(len(os.listdir('./TRAIN/NO')))
print(len(os.listdir('./TRAIN/YES')))
print(len(os.listdir('./TEST/NO')))
print(len(os.listdir('./TEST/YES')))
print(len(os.listdir('./VAL/NO')))
print(len(os.listdir('./VAL/YES')))
print(len(os.listdir('./TEST_BMI/NO')))
print(len(os.listdir('./TEST_BMI/YES')))
print(len(os.listdir('./TEST_BTD/NO')))
print(len(os.listdir('./TEST_BTD/YES')))
print(len(os.listdir('./TEST_dat/NO')))
print(len(os.listdir('./TEST_dat/YES')))

1500
1500
1150
1150
50
50
300
300
500
500
0
0
500
500


In [ ]:
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
TEST_BMI_DIR = 'TEST_BMI/'
TEST_BTD_DIR = 'TEST_BTD/'
TEST_dat_DIR = 'TEST_dat/'
IMG_SIZE = (224,224)
from data_utils import load_data

# use predefined function to load the image data into workspace
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)
X_test_bmi, y_test_bmi, _ = load_data(TEST_BMI_DIR, IMG_SIZE)
X_test_btd, y_test_btd, _ = load_data(TEST_BTD_DIR, IMG_SIZE)
X_test_dat, y_test_dat, _ = load_data(TEST_dat_DIR, IMG_SIZE)

ModuleNotFoundError: ignored